In [1]:
import os

from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
import pickle
import pandas as pd

import jams
import numpy as np

import muda

In [2]:
def root(x):
    return os.path.splitext(os.path.basename(x))[0]

ANNOS = jams.util.find_with_extension('chordlab', 'jams')
AUDIO = jams.util.find_with_extension('data', 'flac')

# Make sure there are the same number of files
assert len(AUDIO) == len(ANNOS)

# And that they're in agreement
assert all([root(_1) == root(_2) for (_1, _2) in zip(AUDIO, ANNOS)])

In [3]:
def augment(afile, jfile, deformer, outpath):
    
    jam = muda.load_jam_audio(jfile, afile)
    
    base = root(afile)
    outfile = os.path.join(outpath, base)
    
    for i, jam_out in enumerate(deformer.transform(jam)):
        muda.save('{}.{}.flac'.format(outfile, i),
                  '{}.{}.jams'.format(outfile, i),
                 jam_out, strict=False)

In [4]:
# Create the augmentation engine for 1-6 half-note
pitcher = muda.deformers.PitchShift(n_semitones=[-1, 1, -2, 2, -3, 3, -4, 4, -5, 5, -6, 6])

In [5]:
with open('working/chords/muda.pkl', 'wb') as fd:
    pickle.dump(pitcher, fd)

In [6]:
OUT_DIR = 'chordlab/'

In [7]:
Parallel(n_jobs=20, verbose=10)(delayed(augment)(aud, jam, pitcher, OUT_DIR) for (aud, jam) in zip(AUDIO, ANNOS));

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done   2 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done   3 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done   4 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done   5 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done   6 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done   7 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done   8 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done   9 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done  11 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done  12 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done  13 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done  14 tasks      | elapsed:    2.7s
[Parallel(n_jobs=20)]: Done  15 tasks      | elapsed:  

ParameterError: Unknown JAMS extension format: "lab"

In [ ]:
# # Build a pump
# sr = 44100
# hop_length = 4096

# p_feature = pumpp.feature.CQTMag(name='cqt', sr=sr, hop_length=hop_length, log=True, conv='tf', n_octaves=6)
# p_chord_tag = pumpp.task.ChordTagTransformer(name='chord_tag', sr=sr, hop_length=hop_length, sparse=True)
# p_chord_struct = pumpp.task.ChordTransformer(name='chord_struct', sr=sr, hop_length=hop_length, sparse=True)

# pump = pumpp.Pump(p_feature, p_chord_tag, p_chord_struct)

# # Save the pump

# with open('/home/bmcfee/working/chords/pump.pkl', 'wb') as fd:
#     pickle.dump(pump, fd)